In [26]:
# Ref: https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd
# Import the necessary libraries

import collections
import os
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

# load the data set

def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

# Updates internal vocabulary based on a list of texts.

def tokenize(x):
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

english_sentences = load_data('smallvocab_en.txt') # Download the input vocabulary
french_sentences = load_data('smallvocab_fr.txt')  # Download the output vocabulary

# Pad each sentence

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post') # Pads sequences to the same length.

# Preproceesing the data

def preprocess(x, y):
    
    # First tokenize
    
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    
    # Pad the vectors
    
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
   
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, y_tk

# Process the input and output vocabulary

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)



def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

# The encode decode model

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  
    learning_rate = 1e-3
    model = Sequential()
    # LSTM model takes a lot of time. So GRU is used
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length)) # Repeats the vector
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax'))) # Applies a layer
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model

# This is for testing the encode-decode model

def test_encdec_model(encdec_model):
    input_shape = (137861, 15, 1)
    output_sequence_length = 21
    english_vocab_size = 199
    french_vocab_size = 344
    model = encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size)


tmp_x = pad(preproc_english_sentences) # Full complete preprocessed input
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1)) # We need to reshape it for the above functions to work well

encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2) # Trains the model
print(logits_to_text(encodeco_model.predict(tmp_x[:2])[0], french_tokenizer)) 



Epoch 1/20
108/108 [==============================] - 91s 825ms/step - loss: 2.9967 - accuracy: 0.4386 - val_loss: 2.4427 - val_accuracy: 0.4877
Epoch 2/20
108/108 [==============================] - 93s 866ms/step - loss: 2.3153 - accuracy: 0.4999 - val_loss: 2.2282 - val_accuracy: 0.5065
Epoch 3/20
108/108 [==============================] - 93s 859ms/step - loss: 2.1253 - accuracy: 0.5102 - val_loss: 1.9606 - val_accuracy: 0.5166
Epoch 4/20
108/108 [==============================] - 91s 841ms/step - loss: 1.8480 - accuracy: 0.5461 - val_loss: 1.7557 - val_accuracy: 0.5634
Epoch 5/20
108/108 [==============================] - 91s 845ms/step - loss: 1.7004 - accuracy: 0.5690 - val_loss: 1.6456 - val_accuracy: 0.5756
Epoch 6/20
108/108 [==============================] - 93s 859ms/step - loss: 1.6081 - accuracy: 0.5814 - val_loss: 1.5678 - val_accuracy: 0.5882
Epoch 7/20
108/108 [==============================] - 93s 858ms/step - loss: 1.5418 - accuracy: 0.5899 - val_loss: 1.5112 - val_ac

In [27]:
import googletrans
from langdetect import detect
from googletrans import Translator  

#simple function to detect and translate text 
def detect_and_translate(text,target_lang):
    
    result_lang = detect(text)
    
    if result_lang == target_lang:
        return text 
    
    else:
        translator = Translator()
        translate_text = translator.translate(text,dest = target_lang)
        return translate_text 

sentence = "La capitale dell'India è Delhi"

print(detect_and_translate(sentence,target_lang='en'))

Translated(src=it, dest=en, text=The capital of India is Delhi, pronunciation=None, extra_data="{'confiden...")
